In [1]:
import math

In [2]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense, Input
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.
/usr/local/lib/python3.5/dist-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 6021 on context None
Preallocating 3633/4037 Mb (0.900000) on cuda1
Mapped name None to device cuda1: GeForce GTX 970 (0000:02:00.0)


In [3]:
input_tensor = Input(shape=(3, 400, 400))
base_model = InceptionV3(weights='imagenet', include_top=False, 
                         input_tensor=input_tensor)

In [4]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [5]:
model = Model(inputs=base_model.input, outputs=predictions)

In [6]:
for layer in base_model.layers:
    layer.trainable = False

In [7]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
path = 'data/'

In [9]:
gen = ImageDataGenerator(horizontal_flip=True, height_shift_range=0.1, 
                         width_shift_range=0.1, rotation_range=10)
batches = gen.flow_from_directory(path + 'train', target_size=(400, 400))
valid_batches = gen.flow_from_directory(path + 'valid', target_size=(400, 400))

Found 2067 images belonging to 2 classes.
Found 228 images belonging to 2 classes.


In [10]:
batch_size = 32

In [11]:
model.load_weights(path + 'inception.h5')

In [12]:
model.fit_generator(batches, math.ceil(batches.samples/batch_size), 
                    epochs=15, validation_data=valid_batches, 
                    validation_steps=math.ceil(valid_batches.samples/batch_size))

Epoch 1/15
65/65 [==============================] - 67s - loss: 0.1296 - acc: 0.9612 - val_loss: 0.3729 - val_acc: 0.9123
Epoch 2/15
65/65 [==============================] - 55s - loss: 0.0952 - acc: 0.9654 - val_loss: 0.1772 - val_acc: 0.9561
Epoch 3/15
65/65 [==============================] - 60s - loss: 0.1395 - acc: 0.9543 - val_loss: 0.1491 - val_acc: 0.9561
Epoch 4/15
65/65 [==============================] - 58s - loss: 0.1220 - acc: 0.9591 - val_loss: 0.2003 - val_acc: 0.9430
Epoch 5/15
65/65 [==============================] - 59s - loss: 0.1303 - acc: 0.9551 - val_loss: 0.6413 - val_acc: 0.8553
Epoch 6/15
65/65 [==============================] - 54s - loss: 0.1089 - acc: 0.9615 - val_loss: 0.2045 - val_acc: 0.9430
Epoch 7/15
65/65 [==============================] - 55s - loss: 0.1226 - acc: 0.9611 - val_loss: 0.0749 - val_acc: 0.9737
Epoch 8/15
65/65 [==============================] - 59s - loss: 0.1096 - acc: 0.9554 - val_loss: 0.1083 - val_acc: 0.9649
Epoch 9/15
65/65 [======

In [13]:
for layer in model.layers[:249]:
    layer.trainable = False

for layer in model.layers[249:]:
    layer.trainable = True

In [14]:
from keras.optimizers import SGD

In [15]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.fit_generator(batches, int(batches.samples/batch_size), 
                    epochs=15, validation_data=valid_batches, 
                    validation_steps=int(valid_batches.samples/batch_size))

Epoch 1/15
64/64 [==============================] - 62s - loss: 0.0565 - acc: 0.9780 - val_loss: 0.0787 - val_acc: 0.9796
Epoch 2/15
64/64 [==============================] - 55s - loss: 0.0698 - acc: 0.9753 - val_loss: 0.1202 - val_acc: 0.9541
Epoch 3/15
64/64 [==============================] - 55s - loss: 0.0379 - acc: 0.9844 - val_loss: 0.1186 - val_acc: 0.9490
Epoch 4/15
64/64 [==============================] - 55s - loss: 0.0513 - acc: 0.9811 - val_loss: 0.1266 - val_acc: 0.9847
Epoch 5/15
64/64 [==============================] - 55s - loss: 0.0477 - acc: 0.9810 - val_loss: 0.1964 - val_acc: 0.9643
Epoch 6/15
64/64 [==============================] - 55s - loss: 0.0323 - acc: 0.9873 - val_loss: 0.0955 - val_acc: 0.9541
Epoch 7/15
64/64 [==============================] - 55s - loss: 0.0352 - acc: 0.9849 - val_loss: 0.0710 - val_acc: 0.9898
Epoch 8/15
64/64 [==============================] - 55s - loss: 0.0453 - acc: 0.9819 - val_loss: 0.0985 - val_acc: 0.9694
Epoch 9/15
64/64 [======

In [17]:
model.save_weights(path + 'inception.h5')

In [18]:
for layer in model.layers[200:]:
    layer.trainable = True

In [19]:
model.fit_generator(batches, math.ceil(batches.samples/batch_size), 
                    epochs=15, validation_data=valid_batches, 
                    validation_steps=math.ceil(valid_batches.samples/batch_size))

Epoch 1/15
65/65 [==============================] - 63s - loss: 0.0280 - acc: 0.9913 - val_loss: 0.1964 - val_acc: 0.9561
Epoch 2/15
65/65 [==============================] - 56s - loss: 0.0456 - acc: 0.9814 - val_loss: 0.1474 - val_acc: 0.9649
Epoch 3/15
65/65 [==============================] - 56s - loss: 0.0331 - acc: 0.9885 - val_loss: 0.1233 - val_acc: 0.9561
Epoch 4/15
65/65 [==============================] - 56s - loss: 0.0331 - acc: 0.9894 - val_loss: 0.1717 - val_acc: 0.9561
Epoch 5/15
65/65 [==============================] - 56s - loss: 0.0327 - acc: 0.9865 - val_loss: 0.0565 - val_acc: 0.9781
Epoch 6/15
65/65 [==============================] - 56s - loss: 0.0309 - acc: 0.9881 - val_loss: 0.1840 - val_acc: 0.9430
Epoch 7/15
65/65 [==============================] - 56s - loss: 0.0312 - acc: 0.9889 - val_loss: 0.0937 - val_acc: 0.9649
Epoch 8/15
65/65 [==============================] - 56s - loss: 0.0364 - acc: 0.9877 - val_loss: 0.0836 - val_acc: 0.9693
Epoch 9/15
65/65 [======

In [20]:
for layer in model.layers[100:]:
    layer.trainable = True

In [21]:
model.fit_generator(batches, math.ceil(batches.samples/batch_size), 
                    epochs=15, validation_data=valid_batches, 
                    validation_steps=math.ceil(valid_batches.samples/batch_size))

Epoch 1/15
65/65 [==============================] - 63s - loss: 0.0228 - acc: 0.9913 - val_loss: 0.1634 - val_acc: 0.9474
Epoch 2/15
65/65 [==============================] - 56s - loss: 0.0298 - acc: 0.9885 - val_loss: 0.0849 - val_acc: 0.9737
Epoch 3/15
65/65 [==============================] - 56s - loss: 0.0265 - acc: 0.9928 - val_loss: 0.1849 - val_acc: 0.9561
Epoch 4/15
65/65 [==============================] - 56s - loss: 0.0313 - acc: 0.9896 - val_loss: 0.1393 - val_acc: 0.9518
Epoch 5/15
65/65 [==============================] - 56s - loss: 0.0297 - acc: 0.9899 - val_loss: 0.0742 - val_acc: 0.9737
Epoch 6/15
65/65 [==============================] - 56s - loss: 0.0246 - acc: 0.9913 - val_loss: 0.1490 - val_acc: 0.9649
Epoch 7/15
65/65 [==============================] - 56s - loss: 0.0327 - acc: 0.9891 - val_loss: 0.0898 - val_acc: 0.9781
Epoch 8/15
65/65 [==============================] - 56s - loss: 0.0325 - acc: 0.9865 - val_loss: 0.1358 - val_acc: 0.9605
Epoch 9/15
65/65 [======

In [22]:
model.save_weights(path + 'inception.h5')

In [23]:
for layer in model.layers[0:]:
    layer.trainable = True

In [24]:
model.fit_generator(batches, math.ceil(batches.samples/batch_size), 
                    epochs=15, validation_data=valid_batches, 
                    validation_steps=math.ceil(valid_batches.samples/batch_size))

Epoch 1/15
65/65 [==============================] - 64s - loss: 0.0320 - acc: 0.9889 - val_loss: 0.1069 - val_acc: 0.9649
Epoch 2/15
65/65 [==============================] - 56s - loss: 0.0182 - acc: 0.9937 - val_loss: 0.1034 - val_acc: 0.9649
Epoch 3/15
65/65 [==============================] - 56s - loss: 0.0271 - acc: 0.9881 - val_loss: 0.1257 - val_acc: 0.9649
Epoch 4/15
65/65 [==============================] - 56s - loss: 0.0257 - acc: 0.9913 - val_loss: 0.1311 - val_acc: 0.9649
Epoch 5/15
65/65 [==============================] - 56s - loss: 0.0168 - acc: 0.9952 - val_loss: 0.1407 - val_acc: 0.9561
Epoch 6/15
65/65 [==============================] - 56s - loss: 0.0255 - acc: 0.9933 - val_loss: 0.0890 - val_acc: 0.9781
Epoch 7/15
65/65 [==============================] - 56s - loss: 0.0202 - acc: 0.9925 - val_loss: 0.1324 - val_acc: 0.9693
Epoch 8/15
65/65 [==============================] - 56s - loss: 0.0240 - acc: 0.9894 - val_loss: 0.1278 - val_acc: 0.9737
Epoch 9/15
65/65 [======

In [25]:
model.save_weights(path + 'inception.h5')

## Create submission

In [ ]:
import math
import os
import numpy as np
from glob import glob

In [ ]:
%cd data/test/
%mkdir unknown/

In [ ]:
g = glob('*.jpg')
for img in g:
    os.rename(img, 'unknown/' + img)

In [ ]:
%cd ../..

In [ ]:
test_batches = ImageDataGenerator().flow_from_directory(path + 'test', 
                                                        shuffle=False, 
                                                        batch_size=batch_size,
                                                        class_mode=None,
                                                        target_size=(300, 300))
preds = model.predict_generator(test_batches, math.ceil(test_batches.samples/batch_size))
filenames = test_batches.filenames
indices = [name[8:name.find('.')] for name in filenames]

In [ ]:
subm = np.stack([indices, preds[:,1]], axis=1)

In [ ]:
np.savetxt('data/preds.csv', subm, header='name,invasive', comments='', fmt='%s,%s')